# Inferencia directa con metodos de prompteo usando Langchain - OpenAI

In [45]:
import os
import json
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from dotenv import load_dotenv
from openai import AzureOpenAI
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
    confusion_matrix,
)


In [3]:
# Cargar variables de entorno
load_dotenv(override=True)

AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")
AZURE_KEY = os.getenv("AZURE_OPENAI_KEY")
DEPLOYMENT = os.getenv("DEPLOYMENT")
BING_KEY = os.getenv("BING_KEY")

API_VERSION = "2025-03-01-preview"

# crear el cliente de openai
client = AzureOpenAI(
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_KEY,
)

In [ ]:
def create_prompt(question:str, answers:list[str]):
    """
    Crea el prompt para el modelo de lenguaje con la pregunta y las posibles respuestas.
    Args:
    - question (str): La pregunta a responder.
    - answers (list[str]): Lista de posibles respuestas.
    Returns:
    - prompt (str): El prompt formateado para el modelo.
    - letter2answer (dict): Diccionario que mapea letras a respuestas.
    """
    if answers[4] == "Respuesta incorrecta":
        answers = answers[:-1]
    letters = ['A', 'B', 'C', 'D', 'E'][:len(answers)]

    options_text = "\n".join(f"{letters[i]}) {answers[i]}" for i in range(len(answers)))

    letter2answer = {letters[i]: answers[i] for i in range(len(answers))}


    return f"""
Pregunta:
{question}

Opciones:
{options_text}

Respuesta (solo la letra):
""", letter2answer

Leer los datos de test.

In [7]:
df_test = pd.read_csv('data/test.csv', index_col=0)
df_test.head()

,qid,qtext,ra,image,answer_1,answer_2,answer_3,answer_4,answer_5
4039,87,El virión de los retrovirus:,4,NaN,Tiene forma helicoidal.,Tiene forma icosaédrica.,Contiene una sola copia de su genoma.,Contiene dos copias de su genoma.,Contiene un genoma segmentado.
4173,224,El suelo de la cavidad amniótica es el:,3,NaN,Trofoblasto.,Hipoblasto.,Epiblasto.,Endometrio.,Miometrio.
3975,18,Las señales sensitivas llegan principalmente a...,3,NaN,II.,III.,IV.,V.,VI.
2470,25,Las desviaciones instrumentales de la Ley de B...,2,NaN,Variaciones en la temperatura lo que provoca d...,Empleo de radiación no monocromática y presenc...,"Empleo de concentraciones elevadas de analito,...",La participación de la especie absorbente en u...,Respuesta incorrecta
2255,36,La presencia de síntomas o déficits que afecta...,3,NaN,Trastorno somatomorfo indiferenciado.,Trastorno de somatización.,Trastorno de conversión.,Trastorno por dolor.,Respuesta incorrecta


Ejemplo de respuesta a pregunta.

In [99]:
example = df_test.iloc[215]
question = example["qtext"]
answers = [
    example["answer_1"],
    example["answer_2"],
    example["answer_3"],
    example["answer_4"],
    example["answer_5"],
]
real_answer = answers[example["ra"] - 1]

# crear el prompt
prompt, l2a = create_prompt(question, answers)
print(prompt)


Pregunta:
Las funciones del Box de Triaje son varias, EXCEPTO:

Opciones:
A) Clasificación de los pacientes a fin de priorizar la atención urgente según su gravedad.
B) Determinar el tiempo de atención y el recurso más adecuado en cada caso.
C) Precisar el diagnóstico médico.
D) Permitir el trabajo simultáneo de dos profesionales en situación de pico de demanda asistencial.

Respuesta (solo la letra):



Se realiza una prueba con el modelo de OpenAI gpt-4o-mini.

In [100]:
response = client.chat.completions.create(
    model=DEPLOYMENT,
    messages=[
        {"role": "system", "content": "Eres un examinador experto en ambitos biomedicos.\nDebes seleccionar la opción respuesta correcta entre las opciones que se te indiquen.\nResponde únicamente con UNA letra (A, B, C, D o E) según corresponda.\nPiensa paso a paso."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=2000,
    temperature=0.3
)
resp = response.choices[0].message.content
print(f'respuesta correcta: {real_answer}')
print(resp)
print(f'respuesta predicha: {l2a[resp]}')

respuesta correcta: Precisar el diagnóstico médico.
C
respuesta predicha: Precisar el diagnóstico médico.


Procedemos a procesar todo el dataset

In [ ]:
def obtain_answer(pregunta:str, respuestas:list[str]):
    """
    Obtiene la respuesta del modelo de lenguaje dado una pregunta y posibles respuestas.
    Args:
    - pregunta (str): La pregunta a responder.
    - respuestas (list[str]): Lista de posibles respuestas.
    Returns:
    - dict: Diccionario con la respuesta obtenida, incluyendo la letra, texto y índice.
    """
    # crear el prompt
    user_prompt, l2a = create_prompt(pregunta, respuestas)
    # print(user_prompt)
    # diccionario para traducir letra a un int
    l2int = {'A':1, 'B':2, 'C':3, 'D':4,'E':5}

    # pasarlo a openai
    response = client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[
            {"role": "system", "content": "Eres un examinador experto en ambitos biomedicos.\nDebes seleccionar la opción respuesta correcta entre las opciones que se te indiquen.\nResponde únicamente con UNA letra (A, B, C, D o E) según corresponda.\nPiensa paso a paso."},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=200,
        temperature=0.3
    )

    # obtener la respuesta cruda
    raw = response.choices[0].message.content
    # print(raw)

    # extraer la letra (A–E), soporta casos como 'La respuesta correcta es B'
    match = re.search(r"[A-E]", raw, flags=re.IGNORECASE)
    if match:
        letter = match.group(0).upper()
    else:
        # Respuesta inválida
        return {
            "raw": raw,
            "letter": None,
            "answer_text": None,
            "answer_index": -1
        }
    
    # Mapear a texto de respuesta
    answer_text = l2a.get(letter, None)
    answer_index = l2int.get(letter, -1)

    return {
        "raw": raw,
        "letter": letter,
        "answer_text": answer_text,
        "answer_index": answer_index
    }

In [ ]:
def evaluar(test_data:pd.DataFrame):
    """
    Evalúa el rendimiento del modelo en un conjunto de datos de prueba.
    Args:
    - test_data (pd.DataFrame): DataFrame con los datos de prueba.
    Returns:
    - dict: Diccionario con las métricas de evaluación.
    """
    all_preds = []
    all_labels = []

    total = len(test_data)

    for i, row in tqdm(test_data.iterrows(), total=total, desc="Evaluating"):
        question = row["qtext"]
        answers = [
            row["answer_1"],
            row["answer_2"],
            row["answer_3"],
            row["answer_4"],
            row["answer_5"],
        ]
        real_answer = row["ra"]  # 1–5
        all_labels.append(real_answer)

        # obtener prediccion
        ans = obtain_answer(question, answers)
        all_preds.append(ans['answer_index'])
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    # print(all_preds)
    # print(all_labels)

    # Metricas
    valid_mask = all_preds != -1
    invalid_predictions = np.sum(all_preds == -1)

    print(f"Predicciones inválidas (fuera de opciones): {invalid_predictions}")

    if invalid_predictions > 0:
        print("Nota: las predicciones inválidas se excluyen de métricas de F1/precision/recall.")

    metrics = {
        "Accuracy": np.mean(all_preds == all_labels),
        "F1 Macro": f1_score(all_labels[valid_mask], all_preds[valid_mask], average='macro'),
        "F1 Micro": f1_score(all_labels[valid_mask], all_preds[valid_mask], average='micro'),
        "F1 Weighted": f1_score(all_labels[valid_mask], all_preds[valid_mask], average='weighted'),

        "Precision Macro": precision_score(all_labels[valid_mask], all_preds[valid_mask], average='macro'),
        "Precision Micro": precision_score(all_labels[valid_mask], all_preds[valid_mask], average='micro'),
        "Precision Weighted": precision_score(all_labels[valid_mask], all_preds[valid_mask], average='weighted'),

        "Recall Macro": recall_score(all_labels[valid_mask], all_preds[valid_mask], average='macro'),
        "Recall Micro": recall_score(all_labels[valid_mask], all_preds[valid_mask], average='micro'),
        "Recall Weighted": recall_score(all_labels[valid_mask], all_preds[valid_mask], average='weighted'),
    }

    print("Resultados de la evaluación:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

    print("Classification Report:")
    print(classification_report(all_labels[valid_mask], all_preds[valid_mask]))

    cm = confusion_matrix(all_labels[valid_mask], all_preds[valid_mask])
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    labels = [1, 2, 3, 4, 5]  # clases 1–5

    plt.figure(figsize=(6, 5))
    sns.heatmap(
        cm_normalized, annot=True, fmt=".2f", cmap="Blues",
        xticklabels=labels, yticklabels=labels
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Normalized Confusion Matrix")
    plt.tight_layout()
    plt.show()

    return metrics


In [101]:
metrics = evaluar(df_test)

Evaluating:  27%|██▋       | 214/790 [01:37<04:22,  2.19it/s]


BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': True, 'severity': 'high'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}

*Nota: no se finaliza la evaluación ya que OpenAI no permite generación en ciertos contextos, como self-harm, y existen preguntas relacionadas a esto en el dataset :disappointed: